# Performance


*Avoiding slow code*

With pandas, you'll get the most bang for your performance-buck by avoiding antipatterns. Once you've done that there are additional options like using Numba or Cython if you really need to optimize a piece of code, but that's more work typically.

This notebook will walk through several common mistakes, and show more performant ways of achieving the same thing.

In [1]:
import numpy as np
import pandas as pd

## Mistake 1: Using pandas

pandas isn't always the right choice. If you're dealing with non-tabular data, or lots of linear algebra, you might be better off using something else like Python lists / dicts / sets, or raw NumPy arrays.

## Mistake 2: Using object dtype

Jake VanderPlas has a [great article](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/) on why Python is slow for many of the things we care about as analysts / scientists. One reason is the overhead that comes from  using python objects for integers, floats, etc. relative to their native versions in languages like C.

As a small demonstration, we'll make two NumPy arrays, one with python integers, and one with NumPy's int64.

In [7]:
# Two arrays, different dtypes
s1 = np.arange(10000, dtype=object)
s2 = np.arange(10000, dtype=np.int64)

Now let's do a simple operation on them, like taking the sum.

In [5]:
%timeit s1.sum()

215 µs ± 4.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit s2.sum()

5.56 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


NumPy can process the specialized int64 dtype array faster than the python object version, even though they're equal. Part of this comes from the different algorithms (the NumPy version would overflow with very large integers), and part comes from the Python version having to repeated unbox the actual integer from the Python object, and re-box it for the result.

Typically you would never explicitly pass in dtype=object there, but occasionally object dtypes slip into pandas

* Reading messy Excel Files / CSV files

  These file types either don't have or don't enforce types. pandas has to infer dtypes, which doesn't always go as expected

* "Exotic" data types like Dates, Times, Decimals

    Pandas has implemented a specialized version of datetime.datime, and datetime.timedelta, but not datetime.date, datetime.time, Decimal, etc. Depending on your application, you might be able to treat dates as datetimess, at midnight.

As discussed in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/basics.html#dtypes), pandas mostly uses NumPy's data types. Recent versions include more *extension types*, which are non-NumPy dtypes inside a Series or a DataFrame. Right now the most popular are

* Datetimes with Timezones
* Categorical
* StringDtype
* Nullable Integer
* Nullable Boolean

Previously, pandas couldn't natively store an array of integers with some missing values, since we used `np.nan` as a missing value indicator. `np.nan` is a float, and an array of some integers and some floats is just cast to an array of floats.

In [8]:
pd.Series([1, None, 2])

0    1.0
1    NaN
2    2.0
dtype: float64

You might have explicitly requested `dtype=object` to keep it from being cast to float.

In [9]:
pd.Series([1, None, 2], dtype=object)

0       1
1    None
2       2
dtype: object

But as we know, object-dtype is slow. It's better to use pandas nullable integer dtype:

In [10]:
pd.Series([1, None, 2], dtype=pd.Int64Dtype())

0       1
1    <NA>
2       2
dtype: Int64

In [11]:
a = pd.Series([None] + list(range(10000)), dtype=object)
%timeit a.mean()

649 µs ± 41.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
b = pd.Series([None] + list(range(10000)), dtype=pd.Int64Dtype())
%timeit b.mean()

139 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


If you have some messy data in-memory (in a Python list, say) that you wish to convert, use one of pandas' parsers like

- `pd.to_numeric`
- `pd.to_datetime`
- `pd.to_timedelta`

Or use `DataFrame.convert_dtypes()` to convert from the "old" versions (e.g. floats with NaNs) to the new nullable versions (e.g. nullable integer).

### Categoricals

pandas has a [Categorical Data Type](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Categorical.html) that represents data coming from a specific, and typically *fixed* set of values. It's data model includes two components:

1. `categories`: An Index storing the set of valid values.
2. `ordered`: A boolean flag indicating whether there's an ordering between the values.

If you try to set a value that's not contained in the `categories`, you'll see an exception.

In [13]:
a = pd.Categorical(['good', 'better', 'good', 'best'],
                   categories=['good', 'better', 'best'],
                   ordered=True)
a

['good', 'better', 'good', 'best']
Categories (3, object): ['good' < 'better' < 'best']

In [14]:
a[0] = 'OK'

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

While the primary intent of categoricals was to model data with a fixed set of allowed values, their implementation suggests another use: saving memory and improving performance.

When you make a Categorical, pandas will *factorize* the values. This creates a mapping between the original value and an integer code.

In [15]:
a

['good', 'better', 'good', 'best']
Categories (3, object): ['good' < 'better' < 'best']

In [16]:
a.categories

Index(['good', 'better', 'best'], dtype='object')

In [17]:
a.codes

array([0, 1, 0, 2], dtype=int8)

Notice two things:

1. The value `"good"` is associated with code `0`, the first element in `.categories`.
2. The codes are stored as `int8`.

When you have many repeated values, whose regular representation are larger than int8, then storing the data as Categorical can have some performance benefits.

To demonstrate this, let's suppose you had a table with every adult resident of the United States (about 321,000,000 rows) where one column stores the state abbreviation as a string.

Just storing this as an object-dtype array would cost the size of the 2-character string times the 321,000,000 occurances:

In [20]:
import sys

population = 321_000_000
bytes_per_person = sys.getsizeof("AL")  # two characters per state
nbytes = population * bytes_per_person
print("{:,d} MB".format(nbytes // 1_000_000))

16,371 MB


Or about 16GB. How many MB would you need to store the same as a Categorical?

In [27]:
character_bytes = 50 * sys.getsizeof("AL")  # store all the states onces
bytes_per_person = 1  # How many bytes does each row take? Which array is this? Check `.dtype.itemsize`
# 
nbytes = (character_bytes + (population * bytes_per_person))

print("{:,d} MB".format(nbytes // 1_000_000))

321 MB


In [28]:
# %load solutions/performance_categorical.py

character_bytes = 50 * sys.getsizeof("AL")  # Store all the states once
bytes_per_person = 1  # np.int8 = 1 byte per element
nbytes = (character_bytes + (population * bytes_per_person))
print("{:,d} MB".format(nbytes // 1_000_000))


321 MB


So when there are many repeated values the savings can be dramatic.

## Vectorization

Just like with NumPy, *vectorization* is key to getting good performance out of pandas.
**The short definition is "don't do for loops", the longer definition is "let NumPy do the for loop in C".**

As an example, let's grab some data on airports locations. We'll compote the distances between pairs of airports.

In [29]:
airports = pd.read_csv(
    "https://vega.github.io/vega-datasets/data/airports.csv",
    index_col="iata",
    nrows=500,
)
airports

,name,city,state,country,latitude,longitude
iata,,,,,,
00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...
57B,Islesboro,Islesboro,ME,USA,44.302856,-68.910587
57C,East Troy Municipal,East Troy,WI,USA,42.797111,-88.372500
59B,Newton,Jackman,ME,USA,45.631991,-70.247289


This next block is a bit of pandas magic to build a DataFrame with each pair.

In [30]:
columns = ["longitude", "latitude"]
idx = pd.MultiIndex.from_product([airports.index, airports.index],
                                 names=['orig', 'dest'])

pairs = pd.concat([
    airports[columns]
        .rename(columns = lambda x: f'{x}_orig')
        .reindex(idx, level='orig'),
    airports[columns]
        .rename(columns = lambda x: f'{x}_dest')
        .reindex(idx, level='dest')
    ], axis="columns"
)
pairs

longitude_orig  latitude_orig  longitude_dest  latitude_dest
orig dest                                                              
00M  00M       -89.234505      31.953765      -89.234505      31.953765
     00R       -89.234505      31.953765      -95.017928      30.685861
     00V       -89.234505      31.953765     -104.569893      38.945749
     01G       -89.234505      31.953765      -78.052081      42.741347
     01J       -89.234505      31.953765      -81.905944      30.688012
...                   ...            ...             ...            ...
5A6  57B       -89.729248      33.465401      -68.910587      44.302856
     57C       -89.729248      33.465401      -88.372500      42.797111
     59B       -89.729248      33.465401      -70.247289      45.631991
     5A4       -89.729248      33.465401      -88.726189      34.015805
     5A6       -89.729248      33.465401      -89.729248      33.465401

[250000 rows x 4 columns]

In [31]:
import math


def gcd_py(lat1, lng1, lat2, lng2):
    '''
    Calculate great circle distance between two points.
    https://www.johndcook.com/blog/python_longitude_latitude/

    Parameters
    ----------
    lat1, lng1, lat2, lng2: float

    Returns
    -------
    distance:
      distance from ``(lat1, lng1)`` to ``(lat2, lng2)`` in kilometers.
    '''
    degrees_to_radians = math.pi / 180.0
    ϕ1 = (90 - lat1) * degrees_to_radians
    ϕ2 = (90 - lat2) * degrees_to_radians

    θ1 = lng1 * degrees_to_radians
    θ2 = lng2 * degrees_to_radians

    cos = (math.sin(ϕ1) * math.sin(ϕ2) * math.cos(θ1 - θ2) +
           math.cos(ϕ1) * math.cos(ϕ2))
    # round to avoid precision issues on identical points causing ValueErrors
    cos = round(cos, 8)
    arc = math.acos(cos)
    return arc * 6373  # radius of earth, in kilometers

In [32]:
def gcd_vec(lat1, lng1, lat2, lng2):
    '''
    Calculate great circle distance.
    https://www.johndcook.com/blog/python_longitude_latitude/

    Parameters
    ----------
    lat1, lng1, lat2, lng2: float or array of float

    Returns
    -------
    distance:
      distance from ``(lat1, lng1)`` to ``(lat2, lng2)`` in kilometers.
    '''
    ϕ1 = np.deg2rad(90 - lat1)
    ϕ2 = np.deg2rad(90 - lat2)

    θ1 = np.deg2rad(lng1)
    θ2 = np.deg2rad(lng2)

    cos = (np.sin(ϕ1) * np.sin(ϕ2) * np.cos(θ1 - θ2) +
           np.cos(ϕ1) * np.cos(ϕ2))
    # round to avoid precision issues on identical points causing warnings
    cos = np.round(cos, 8)
    arc = np.arccos(cos)
    return arc * 6373 # radius of earth, in kilometers

In [40]:
%%time
# gcd_py with DataFrame.apply
r = pairs.apply(
    lambda x: gcd_py(x['latitude_orig'],
                     x['longitude_orig'],
                     x['latitude_dest'],
                     x['longitude_dest']),
                axis="columns"
)

r

CPU times: user 4.16 s, sys: 49.6 ms, total: 4.21 s
Wall time: 4.22 s


orig  dest
00M   00M        0.000000
      00R      567.271820
      00V     1589.259385
      01G     1551.898663
      01J      710.296324
                 ...     
5A6   57B     2158.573499
      57C     1044.687644
      59B     2140.173926
      5A4      111.155006
      5A6        0.000000
Length: 250000, dtype: float64

In [42]:
%%time
# gcd_py with manual iteration
r = pd.Series([gcd_py(*x) for x in pairs.itertuples(index=False)],
              index=pairs.index)

r

CPU times: user 489 ms, sys: 10.6 ms, total: 500 ms
Wall time: 499 ms


orig  dest
00M   00M        0.000000
      00R      643.271159
      00V     1705.150273
      01G     1245.424495
      01J      815.177587
                 ...     
5A6   57B     2316.199986
      57C      151.388805
      59B     2167.660415
      5A4      111.570797
      5A6        0.000000
Length: 250000, dtype: float64

In [38]:
%%time
# gcd_vec
r = gcd_vec(pairs['latitude_orig'], pairs['longitude_orig'],
            pairs['latitude_dest'], pairs['longitude_dest'])
r

CPU times: user 166 ms, sys: 10.1 ms, total: 176 ms
Wall time: 173 ms


orig  dest
00M   00M        0.000000
      00R      567.271820
      00V     1589.259385
      01G     1551.898663
      01J      710.296324
                 ...     
5A6   57B     2158.573499
      57C     1044.687644
      59B     2140.173926
      5A4      111.155006
      5A6        0.000000
Length: 250000, dtype: float64

For more, consult these pages from pandas' documentation:

* Enhancing performance: https://pandas.pydata.org/docs/user_guide/enhancingperf.html
* Scaling to larger datasets: https://pandas.pydata.org/docs/user_guide/scale.html

## Constructing DataFrames

pandas DataFrames are somewhat complex objects. They're a collection of many NumPy (or extension) arrays, and index and row labels. Creating them has a decent amount of overhead over a simple NumPy array or Python list.

This can be important when you're collecting many similar files into a single DataFrame. Suppose we had a collection of files

```
data/
  01.json
  02.json
  ...
  99.json
```

Suppose we wanted to read all of these into a single collection. In pure Python, we would probably create an empty list and repeatedly `.append()` to it, perhaps using a list comprehension:

```python
data = []
for path in Path("data").glob("*.json"):
    with path.open() as file:
        file_data = json.load(file)
        data.append(file_data)
```

But with pandas, where creating or growing a DataFrame is much more expensive relative to a Python list, this is slow. Let's run some benchmarks:

In [43]:
import time

size_per = 5000
N = 100
cols = list('abcd')

In [44]:
def timed(n=30):
    '''
    Running a microbenchmark.
    '''
    def deco(func):
        def wrapper(*args, **kwargs):
            timings = []
            for i in range(n):
                t0 = time.time()
                func(*args, **kwargs)
                t1 = time.time()
                timings.append(t1 - t0)
            return timings
        return wrapper
    return deco

In [ ]:
@timed(60)
def append_df():
    '''
    The pythonic (bad) way
    '''
    df = pd.DataFrame(columns=cols)
    for _ in range(N):
        df.append(pd.DataFrame(np.random.randn(size_per, 4), columns=cols))
    return df

@timed(60)
def concat_df():
    '''
    The pandorabe (good) way
    '''
    dfs = [pd.DataFrame(np.random.randn(size_per, 4), columns=cols)
           for _ in range(N)]
    return pd.concat(dfs, ignore_index=True)

In [ ]:
t_append = append_df()
t_concat = concat_df()

timings = (pd.DataFrame({"Append": t_append, "Concat": t_concat})
             .stack()
             .reset_index()
             .rename(columns={0: 'Time (s)',
                              'level_1': 'Method'}))
timings.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(4, 6))
sns.boxplot(x='Method', y='Time (s)', data=timings, ax=ax);

## Using Numba

Recent versions of pandas optionally make extensive use of Numba to speed up certain operations. This is helpful when you have some custom user-defined function that you're passing to one of pandas' `.apply`, `.agg`, or `.transform` methods (in a rolling or groupby context).

Consider something like a `df.rolling(n).apply(func)`. At a high level, that operation requires

1. Splitting the input into groups
2. Applying `func` to each group
3. Collecting the results into an output group


<img src="https://docs.google.com/drawings/d/e/2PACX-1vSpZlYnXg8MfRHlRjm8JDcxkCjrQfI2XoS06JikaoRCuZiQUUgyo5yjWASU-ynNcucK2-eumooIty1-/pub?w=960&amp;h=720">

Now let's suppose we wanted to speed that up with Numba. As a user, you could `@numba.jit` your function. Depending on what your user defined function is doing, that could lead to a nice speedup. But there would still be a bunch of overhead *around* your function that would be relatively slow. Pandas would need to slice into the array (from Python), call your fast function (now in fast machine code), and jump back to Python to form the output array.

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRwvBtrV51LU2qfOxXUrggJ7h0-bTeSSozatQ7AECyhSOxEdO0ivfoXNhwWM5Q-lZvRBxmPMeAX5hzf/pub?w=960&amp;h=540">

When you use the `engine="numba"` keyword, pandas and Numba able to JIT compile a lot more than just your function. We're able to JIT the entire splitting, function application, and result combination so that the whole things stays in fast machine code.

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRYpI3MI4LKZQSz2VUAxQrxiN6wAlnmTCLOF2VcYTDtF5dJEbSE6IY1MgFH8w8GH84Q2Suu9ngjgYD0/pub?w=960&amp;h=540">

For example, let's compute the mean absolute deviation. Pandas doesn't have a builtin version.

In [45]:
def mad(x):
    return np.fabs(x - x.mean()).mean()

In [46]:
ts = pd.DataFrame(np.random.randn(10000, 10))
ts

,0,1,2,3,4,5,6,7,8,9
0,0.613421,-0.219586,-0.386634,-0.448110,-0.438090,2.196406,-0.583671,-0.952862,1.856994,-0.274576
1,-0.381224,-0.364339,-0.250863,-0.309085,-0.147865,-2.031275,-1.640106,0.437354,1.436400,-0.461583
2,-0.316414,-1.248216,-1.296935,-0.151574,1.217946,1.250786,1.463354,-0.678831,-0.364008,-0.674843
3,0.058498,0.233325,-0.366719,-0.352766,1.358817,1.295728,0.230338,-0.268212,-0.275032,-1.718011
4,1.733322,-0.245764,-0.448117,0.000172,-1.094962,0.512798,1.149555,-0.090714,0.435362,0.264017
...,...,...,...,...,...,...,...,...,...,...
9995,-0.618918,0.380435,-1.083644,1.317592,-0.256465,0.288949,-0.802881,0.678735,-0.413475,0.954971
9996,-1.083272,0.417739,0.435016,0.754278,0.802134,0.494616,-1.702661,-1.701113,0.551393,-0.468319
9997,0.185284,0.321419,-0.001564,0.777069,-0.039613,0.088319,-0.270248,-0.257946,1.549968,-1.176371
9998,-0.370743,0.958141,-0.686193,-0.047605,1.478998,-0.414944,-1.378498,-0.961586,0.131339,-0.129489


In [53]:
%time ts.rolling(10).apply(mad, raw=True)

CPU times: user 1.14 s, sys: 11.5 ms, total: 1.15 s
Wall time: 1.15 s


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9995,0.519019,0.910295,0.667338,1.005601,0.332143,0.914619,1.020544,0.741262,1.372706,0.435043
9996,0.490374,0.903286,0.612148,0.816374,0.427457,0.817337,1.049636,0.705926,1.218642,0.459126
9997,0.550433,0.838526,0.557899,0.801970,0.396361,0.676834,0.882858,0.676176,0.959361,0.535229
9998,0.545285,0.763377,0.520668,0.805151,0.556922,0.436043,0.857802,0.658523,0.947262,0.492282


Note that the first time we run with `engine="numba"`, it's not faster, even a little slower. In the next cell we'll run it many times and time it.

In [54]:
%time ts.rolling(10).apply(mad, engine="numba", raw=True)

CPU times: user 50.7 ms, sys: 2.88 ms, total: 53.6 ms
Wall time: 51.3 ms


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9995,0.519019,0.910295,0.667338,1.005601,0.332143,0.914619,1.020544,0.741262,1.372706,0.435043
9996,0.490374,0.903286,0.612148,0.816374,0.427457,0.817337,1.049636,0.705926,1.218642,0.459126
9997,0.550433,0.838526,0.557899,0.801970,0.396361,0.676834,0.882858,0.676176,0.959361,0.535229
9998,0.545285,0.763377,0.520668,0.805151,0.556922,0.436043,0.857802,0.658523,0.947262,0.492282


In [56]:
%timeit ts.rolling(10).apply(mad, engine="numba", raw=True)

43.8 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


I think this this pattern, where a library accepts some user-defined function and JIT-compiles the whole operation with Numba, is gaining popularity. Libraries like [UMAP](https://umap-learn.readthedocs.io/en/latest/) allow users to pass loss functions as numba-jitted functions, enabling customizable loss functions without a loss in performance.